## Constructing matrix representations of circuits

symbolic presenation of circuit operators can be constructed with [sympy](https://www.sympy.org/en/index.html). they can be used to analyse which states a circuit can produce and how this depends on the gate parameters. a function from parameters to final states of circuit can be [lambdyfied](https://docs.sympy.org/latest/modules/utilities/lambdify.html) from the symbolic operator in the other notebooks this approach is used

### General 
A quantum gate operating on one qbit is represented by a $2\times 2$ unitary matrix. In a circuit of $n$ qbits the action of a gate is described by $2^n$-dimensional unitary matrix operators.

In Cirq, a circuit is 'bucketed' into moments, 'A Moment is a collection of Operations that all act during the same abstract time slice'. In particular in one moment each qbit is acted on by at most one gate. The whole moment can be described by a operator matrix. This matrix is obtained as the tensor product of the operators applied to each qbit. If no gate is applied to a qbit, the identity operator will be inserted in the tensor product.  
The whole circuit is then a sequence of moments and the operators for the moments are applied in sequence. In terms of the matrices that means that the matrix operator for the whole circuit is just the matrix product of the operators of all the moments. Note that the left-most moment in the circuit diagram corresponds to the right-most matrix in matrix product notation.

### Use of symbolic matrices
Many gates are parametrized, e.g. with a radian, and to understand the operator of the whole circuit it is useful to see and use the symbolic matrix operator.
This can be done with sympy, by contructing the operator of the circuit from the symbolic operators for each gate. In sympy we can labdify this symbolic matrix, and then have a function from parameters to operators.

### 2 qbits example circuit presented in this notebook

1. one moment circuit is defined with Cirq, including y-rotation gate parametrized by radians $\phi$
2. a symbolic matrix operator is contructed 
3. the matrix is evaluated to check equality to cirq circuit
4. the curcuit is extended by second moment, including y-rotation gate parametrized by radians $\theta$
5. symbolic matrix for second moment is constructed, and evaluated
6. observation that moments can be re-organized


In [26]:
import cirq
import numpy as np
import sympy as sp
from sympy.physics.quantum import TensorProduct

#### define symbols for symbolic matrix and the matrix operators to construct circuit

In [2]:
phi = sp.symbols("phi")
theta = sp.symbols("theta")
# define the four 2x2 operators
yrotation_1 = sp.Matrix([[sp.cos(phi/2),-sp.sin(phi/2)], 
                                [sp.sin(phi/2), sp.cos(phi/2)]])
yrotation_2 = sp.Matrix([[sp.cos(theta/2),-sp.sin(theta/2)], 
                                [sp.sin(theta/2), sp.cos(theta/2)]])
X = sp.Matrix([[0,1], [1,0]])
identity = sp.Matrix([[1,0],[0,1]]) 

#### create circuit with one moment
apply a y-rotation on the first qbit and the X-gate on the second qbit  
value of angle $\phi$ of the y-rotation is set to $0.3 \pi$  
use cirq to define the circuit, print the circuit schema and the unitary operator defined by the circuit

In [3]:
phi_value = 0.3*np.pi
c = cirq.Circuit()
c.append([cirq.ry(phi_value)(cirq.GridQubit(0, 0)),
          cirq.X.on(cirq.GridQubit(0, 1)),
          #cirq.ry(theta_value)(cirq.GridQubit(0, 1))
         ])
print("the circuit:")        
print(c)  
print("has unitary operator matrix:")
sp.Matrix(c.unitary())

the circuit:
(0, 0): ───Ry(0.3π)───

(0, 1): ───X──────────
has unitary operator matrix:


Matrix([
[                0, 0.891006524188368,                  0, -0.453990499739547],
[0.891006524188368,                 0, -0.453990499739547,                  0],
[                0, 0.453990499739547,                  0,  0.891006524188368],
[0.453990499739547,                 0,  0.891006524188368,                  0]])

#### this circuit is composed of a y-rotation...

In [4]:
yrotation_1

Matrix([
[cos(phi/2), -sin(phi/2)],
[sin(phi/2),  cos(phi/2)]])

#### ...and the X-operator on the second qbit

In [5]:
X

Matrix([
[0, 1],
[1, 0]])

#### the operator of the whole circuit 
is then the tensor product of the operator applied to the first qbit and the operator applied to the second qbit (in that order, the tensor product is not communitative)

In [22]:
op_1_symbolic = TensorProduct(yrotation_1, X)
op_1_symbolic

Matrix([
[         0, cos(phi/2),           0, -sin(phi/2)],
[cos(phi/2),          0, -sin(phi/2),           0],
[         0, sin(phi/2),           0,  cos(phi/2)],
[sin(phi/2),          0,  cos(phi/2),           0]])

evaluating this symbolic operator at $\phi = $ phi_value, gives the same operator as we got from cirq circuit

In [13]:
circ_op = sp.lambdify([phi], op_1_symbolic, 'numpy')
sp.Matrix(circ_op(phi_value))

Matrix([
[              0.0, 0.891006524188368,                0.0, -0.453990499739547],
[0.891006524188368,               0.0, -0.453990499739547,                0.0],
[              0.0, 0.453990499739547,                0.0,  0.891006524188368],
[0.453990499739547,               0.0,  0.891006524188368,                0.0]])

#### extending circuit with 2nd moment 

In [23]:
theta_value = 1.5*np.pi
if len(c.moments)==1: # avoid adding several times
    c.append([cirq.ry(theta_value)(cirq.GridQubit(0, 1))])

print("circuit")
print(c)  
print("has unitary operator matrix:")
sp.Matrix(c.unitary())

circuit
(0, 0): ───Ry(0.3π)──────────────

(0, 1): ───X──────────Ry(1.5π)───
has unitary operator matrix:


Matrix([
[-0.630036755335051,  -0.63003675533505,  0.321019760960103,  0.321019760960103],
[ -0.63003675533505,  0.630036755335051,  0.321019760960103, -0.321019760960103],
[-0.321019760960103, -0.321019760960103, -0.630036755335051,  -0.63003675533505],
[-0.321019760960103,  0.321019760960103,  -0.63003675533505,  0.630036755335051]])

#### constructing the operator matrix for that extended circuit
the second moment of the circuit only acts on the second qbit, with an y-rotation with angle $\theta$  
this second moment is described by the tensor product of identity matrix (no action on first qbit) and the y-rotation matrix with parameter $\theta$

In [15]:
op_2_symbolic = TensorProduct(identity, yrotation_2)
op_2_symbolic

Matrix([
[cos(theta/2), -sin(theta/2),            0,             0],
[sin(theta/2),  cos(theta/2),            0,             0],
[           0,             0, cos(theta/2), -sin(theta/2)],
[           0,             0, sin(theta/2),  cos(theta/2)]])

the whole circuit is then the product of the two operators for the first and second moment...

In [16]:
circuit_op = op_2_symbolic*op_1_symbolic
circuit_op

Matrix([
[-sin(theta/2)*cos(phi/2), cos(phi/2)*cos(theta/2),  sin(phi/2)*sin(theta/2), -sin(phi/2)*cos(theta/2)],
[ cos(phi/2)*cos(theta/2), sin(theta/2)*cos(phi/2), -sin(phi/2)*cos(theta/2), -sin(phi/2)*sin(theta/2)],
[-sin(phi/2)*sin(theta/2), sin(phi/2)*cos(theta/2), -sin(theta/2)*cos(phi/2),  cos(phi/2)*cos(theta/2)],
[ sin(phi/2)*cos(theta/2), sin(phi/2)*sin(theta/2),  cos(phi/2)*cos(theta/2),  sin(theta/2)*cos(phi/2)]])

...and evaluating this symbolic operator with parameter $\theta =$ theta_value, gives same unitary matrix and we got from cirq after appending the second moment ot the circuit

In [17]:
circ_op = sp.lambdify([phi, theta], circuit_op, 'numpy')
sp.Matrix(circ_op(phi_value, theta_value))

Matrix([
[-0.630036755335051,  -0.63003675533505,  0.321019760960103,  0.321019760960103],
[ -0.63003675533505,  0.630036755335051,  0.321019760960103, -0.321019760960103],
[-0.321019760960103, -0.321019760960103, -0.630036755335051,  -0.63003675533505],
[-0.321019760960103,  0.321019760960103,  -0.63003675533505,  0.630036755335051]])

#### definition of the two moments in the circuit
In the diagram of the whole circuit, on the first qbit $Ry(\phi)$ is the only operation. in the diagram the gate can move on its line as long as it is not crossing other gates.  
in this example that means, the $Ry(\phi)$ can also be moved left to be part of second moment.

In terms of the matrix operators that means, the identity tensor-factor can move from one matrix-factor to the other, exchanging with the first y-rotation:

In [24]:
(   TensorProduct(identity, yrotation_2)*TensorProduct(yrotation_1, X) 
 == TensorProduct(yrotation_1, yrotation_2)*TensorProduct(identity, X)) 

True